In [34]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import deeplake
from time import time
import pandas as pd


In [35]:
def LoadData(DATA, batch_size):

    if DATA == "CIFAR10":
        root = '../Data/CIFAR10'
        num_classes = 10

        cifar10_train_transform = transforms.Compose([
            transforms.RandomCrop(32),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        cifar10_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

        train = torchvision.datasets.CIFAR10(root=root, train=True, transform=cifar10_train_transform)
        test = torchvision.datasets.CIFAR10(root=root, train=False, transform=cifar10_test_transform)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size,shuffle=False, num_workers=2)
        print(DATA + " successfully loaded.")

    elif DATA == "CIFAR100":
        root = '../Data/CIFAR100'
        num_classes = 100

        cifar100_train_transform = transforms.Compose([
            transforms.RandomResizedCrop(32, scale=(0.8, 1.2)), # Simulate scale variations
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(degrees=15), # Consider for object orientations
            transforms.RandomGrayscale(p=0.2), # Optional for color augmentation
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Standard CIFAR-100 normalization
        ])
        cifar100_test_transform = transforms.Compose([
            # transforms.Resize(32),
            # transforms.CenterCrop(32),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        train = torchvision.datasets.CIFAR100(root=root, train=True, transform=cifar100_train_transform)
        test = torchvision.datasets.CIFAR100(root=root, train=False, transform=cifar100_test_transform)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size,shuffle=False, num_workers=2)
        print(DATA + " successfully loaded.")

    elif DATA == "IMAGENET":
        root = '../Data/IMAGENET'
        num_classes = 200

        tiny_imagenet_train_transform = transforms.Compose([
            transforms.RandomResizedCrop(64, scale=(0.8, 1.2), interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        tiny_imagenet_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])

        train = deeplake.load("hub://activeloop/tiny-imagenet-train")
        test = deeplake.load("hub://activeloop/tiny-imagenet-test")

        train.pytorch(transform = tiny_imagenet_train_transform, batch_size = batch_size, shuffle = True)
        test.pytorch(transform = tiny_imagenet_test_transform, batch_size = batch_size, shuffle = False)
        print(DATA + " successfully loaded.")

    elif DATA == "MNIST":
        root='../Data/MNIST'
        num_classes = 10

        mnist_train_transform = transforms.Compose([
            transforms.RandomRotation(10),  # Improve robustness
            transforms.RandomHorizontalFlip(),  # Augment data
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])


        mnist_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

        train = torchvision.datasets.MNIST(root=root, train=True, transform=mnist_train_transform, download=True)
        test = torchvision.datasets.MNIST(root=root, train=False, transform=mnist_test_transform ,download=True)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
        print(DATA + " successfully loaded.")

    elif DATA == "FASION_MNIST":
        root='../Data/FASION_MNIST'
        num_classes = 10

        fashion_mnist_train_transform = transforms.Compose([
            transforms.RandomRotation(15),  # Improve robustness
            transforms.RandomHorizontalFlip(),  # Augment data
            transforms.RandomResizedCrop(28),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        fashion_mnist_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

        train = torchvision.datasets.FashionMNIST(root=root, train=True, transform=fashion_mnist_train_transform, download=True)
        test = torchvision.datasets.FashionMNIST(root=root, train=False, transform=fashion_mnist_test_transform ,download=True)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
        print(DATA + " successfully loaded.")

    elif DATA == "PascalVOC":
        root = '../Data/PascalVOC'
        num_classes = 20

        voc_train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),
            transforms.RandomHorizontalFlip(),  # Randomly flip for horizontal invariance
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])

        voc_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])

        train = torchvision.datasets.VOCDetection(root=root, year='2012', image_set='trainval', transform=voc_train_transform, download = True)
        test = torchvision.datasets.VOCDetection(root=root, year='2012', image_set='trainval', transform=voc_test_transform, download = True)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
        print(DATA + " successfully loaded.")

    elif DATA == "Flowers102":
        root = '../Data/Flowers102'
        num_classes = 102

        oxford102_train_transform = transforms.Compose([
            transforms.Resize(256),  # Adjust size based on your model
            transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),  # Simulate scale variations
            transforms.RandomHorizontalFlip(),  # Increase data diversity
            transforms.RandomRotation(15),  # Consider for specific flower orientations (optional)
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),  # Introduce color variations
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))  # ImageNet stats, adjust if needed
        ])


        oxford102_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])


        train = torchvision.datasets.Flowers102(root=root, transform=oxford102_train_transform, download=True)
        test = torchvision.datasets.Flowers102(root=root, transform=oxford102_test_transform, download = True)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
        print(DATA + " successfully loaded.")

    return trainloader, testloader, num_classes

In [36]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [38]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()

        self.conv1a = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1a = nn.BatchNorm2d(out_channels)

        self.conv2a = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2a = nn.BatchNorm2d(out_channels)

        self.conv3a = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3a = nn.BatchNorm2d(out_channels*self.expansion)

        self.conv1b = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1b = nn.BatchNorm2d(out_channels)

        self.conv2b = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2b = nn.BatchNorm2d(out_channels)

        self.conv3b = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3b = nn.BatchNorm2d(out_channels*self.expansion)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
        global L

        identity = x.clone()
        xa = self.relu(self.batch_norm1a(self.conv1a(x)))
        xb = self.relu(self.batch_norm1b(self.conv1b(x)))
        x = (xa + xb)/2
        L += torch.mean(torch.pow((xa - x) + (xb - x), 2))

        xa = self.relu(self.batch_norm2a(self.conv2a(x)))
        xb = self.relu(self.batch_norm2b(self.conv2b(x)))
        x = (xa + xb)/2
        L += torch.mean(torch.pow((xa - x) + (xb - x), 2))

        xa = self.batch_norm3a(self.conv3a(x))
        xb = self.batch_norm3b(self.conv3b(x))
        x = (xa + xb)/2
        L += torch.mean(torch.pow((xa - x) + (xb - x), 2))


        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)

        return x

In [39]:
class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)

        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)

        global L
        L = 0

    def forward(self, x):
        global L

        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        l1 = L
        L = 0
        x = self.layer2(x)
        l2 = L
        L = 0
        x = self.layer3(x)
        l3 = L
        L = 0
        x = self.layer4(x)
        l4 = L
        L = 0

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        LL = (l1 + l2 + l3 + l4)/4

        return x, LL

    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []

        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )

        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion

        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))

        return nn.Sequential(*layers)

In [40]:
def ResNet50(num_classes, channels=3):
    return ResNet(Bottleneck, [3,4,6,3], num_classes, channels)

In [41]:
def train(model, dataloader, criterion, optimizer, landaparam):
    total = 0.
    correct = 0.
    running_loss = 0.
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs, LL = model(inputs)
        _, predicted = outputs.max(1)

        base_loss = criterion(outputs, labels)
        loss = base_loss - (landaparam * LL)

        loss.backward()
        optimizer.step()

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        running_loss += loss.item()

    train_accuracy = 100 * correct / total
    train_loss = running_loss / total
    return train_loss, train_accuracy

In [42]:
def test(model, dataloader, criterion, landaparam):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs, LL = model(inputs)

            base_loss = criterion(outputs, labels)
            loss = base_loss - (landaparam * LL)
            total += labels.size(0)

            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            running_loss += loss.item()
    test_accuracy = 100 * correct / total
    test_loss = running_loss / total

    return test_loss, test_accuracy

In [43]:
# Data_List = ["CIFAR10", "CIFAR100", "IMAGENET", "MNIST", "FASION_MNIST", "PascalVOC", "Flowers102"]
Data_List = ["FASION_MNIST"]

for DATA in Data_List:
    trainloader, testloader, num_classes = LoadData(DATA, 128)

    model = ResNet50(num_classes, 1).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
    landaparam = 1e-3

    last_epoch = 0
    num_epochs = 100
    history = []

    for epoch in range(last_epoch,num_epochs):
        t0 = time()
        model.train(True)
        running_loss = 0.0
        correct = 0
        total = 0
        train_loss, train_accuracy = train(model, trainloader, criterion, optimizer, landaparam)
        t1 = time()
        test_loss, test_accuracy = test(model, testloader, criterion, landaparam)
        t2 = time()

        data = {
            "Epoch": epoch + 1,
            "Train Loss": train_loss,
            "Train Accuracy": train_accuracy,
            "Train Time": t1 - t0,
            "Test Loss": test_loss,
            "Test Accuracy": test_accuracy,
            "Test Time": t2 - t1,
        }
        print(data)
        history.append(data)

    pd.DataFrame(history).to_json(f'./History/{DATA}-Neuron-Loss-new.json')

FASION_MNIST successfully loaded.
{'Epoch': 1, 'Train Loss': 0.015421168571710586, 'Train Accuracy': 38.57833333333333, 'Train Time': 108.79418873786926, 'Test Loss': 0.009520590329170226, 'Test Accuracy': 64.4, 'Test Time': 6.596938848495483}
{'Epoch': 2, 'Train Loss': 0.009716101876894633, 'Train Accuracy': 56.28, 'Train Time': 108.50012755393982, 'Test Loss': 0.00647969885468483, 'Test Accuracy': 71.96, 'Test Time': 6.620638370513916}
{'Epoch': 3, 'Train Loss': 0.00844414428671201, 'Train Accuracy': 60.943333333333335, 'Train Time': 107.45290350914001, 'Test Loss': 0.006145624095201492, 'Test Accuracy': 71.03, 'Test Time': 6.612884759902954}
{'Epoch': 4, 'Train Loss': 0.008482145037253698, 'Train Accuracy': 61.718333333333334, 'Train Time': 108.96431946754456, 'Test Loss': 0.00516291557252407, 'Test Accuracy': 75.29, 'Test Time': 6.445012092590332}
{'Epoch': 5, 'Train Loss': 0.007747160944342613, 'Train Accuracy': 64.11333333333333, 'Train Time': 109.06385827064514, 'Test Loss': 0.0